In [1]:
from __future__ import print_function, division
import torch
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import os
import argparse
from time import time
#from load_data import *
from torch.optim import lr_scheduler
#from tool_metrics import bboxes_iou

In [24]:
    class AdaptiveBatchNorm2d(nn.Module):
        def __init__(self, num_features, eps=1e-5, momentum=0.1, affine=True):
            super(AdaptiveBatchNorm2d, self).__init__()
            self.bn = nn.BatchNorm2d(num_features, eps, momentum, affine)
            self.a = nn.Parameter(torch.FloatTensor(1, 1, 1, 1))
            self.b = nn.Parameter(torch.FloatTensor(1, 1, 1, 1))

        def forward(self, x):
            return self.a * x + self.b * self.bn(x)

    class Mish(torch.nn.Module):
        def __init__(self):
            super().__init__()

        def forward(self, x):
            x = x * (torch.tanh(torch.nn.functional.softplus(x)))
            return x

    class wR2_ada(nn.Module):
        def __init__(self, num_classes=1000):
            super(wR2_ada, self).__init__()
            hidden1 = nn.Sequential(
                nn.Conv2d(in_channels=3, out_channels=48, kernel_size=5, padding=2, stride=2),
                AdaptiveBatchNorm2d(48),
                Mish(),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
                nn.Dropout(0.2)
            )
            hidden2 = nn.Sequential(
                nn.Conv2d(in_channels=48, out_channels=64, kernel_size=5, padding=2),
                AdaptiveBatchNorm2d(64),
                Mish(),
                nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
                nn.Dropout(0.2)
            )
            hidden3 = nn.Sequential(
                nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, padding=2),
                AdaptiveBatchNorm2d(128),
                Mish(),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
                nn.Dropout(0.2)
            )
            hidden4 = nn.Sequential(
                nn.Conv2d(in_channels=128, out_channels=160, kernel_size=5, padding=2),
                AdaptiveBatchNorm2d(num_features=160),
                Mish(),
                nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
                nn.Dropout(0.2)
            )
            hidden5 = nn.Sequential(
                nn.Conv2d(in_channels=160, out_channels=192, kernel_size=5, padding=2),
                AdaptiveBatchNorm2d(192),
                Mish(),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
                nn.Dropout(0.2)
            )
            hidden6 = nn.Sequential(
                nn.Conv2d(in_channels=192, out_channels=192, kernel_size=5, padding=2),
                AdaptiveBatchNorm2d(192),
                Mish(),
                nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
                nn.Dropout(0.2)
            )
            hidden7 = nn.Sequential(
                nn.Conv2d(in_channels=192, out_channels=192, kernel_size=5, padding=2),
                AdaptiveBatchNorm2d(192),
                Mish(),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
                nn.Dropout(0.2)
            )
            hidden8 = nn.Sequential(
                nn.Conv2d(in_channels=192, out_channels=192, kernel_size=5, padding=2),
                AdaptiveBatchNorm2d(192),
                Mish(),
                nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
                nn.Dropout(0.2)
            )
            hidden9 = nn.Sequential(
                nn.Conv2d(in_channels=192, out_channels=192, kernel_size=3, padding=1),
                AdaptiveBatchNorm2d(192),
                Mish(),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
                nn.Dropout(0.2)
            )
            hidden10 = nn.Sequential(
                nn.Conv2d(in_channels=192, out_channels=192, kernel_size=3, padding=1),
                AdaptiveBatchNorm2d(192),
                Mish(),
                nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
                nn.Dropout(0.2)
            )

            self.features = nn.Sequential(
                hidden1,
                hidden2,
                hidden3,
                hidden4,
                hidden5,
                hidden6,
                hidden7,
                hidden8,
                hidden9,
                hidden10
            )
            self.classifier = nn.Sequential(
                nn.Linear(23232, 100),
                # nn.ReLU(inplace=True),
                nn.Linear(100, 100),
                # nn.ReLU(inplace=True),
                nn.Linear(100, num_classes),
            )

        def forward(self, x):
            x1 = self.features(x)
            x11 = x1.view(x1.size(0), -1)
            x = self.classifier(x11)
            return x
        
         
    class fh02_ada(nn.Module):
        def __init__(self, num_points, num_classes, wrPath=None):
            super(fh02_ada, self).__init__()
            self.load_wR2(wrPath)
            self.classifier1 = nn.Sequential(
                # nn.Dropout(),
                nn.Linear(53248, 128),
                # nn.ReLU(inplace=True),
                # nn.Dropout(),
                nn.Linear(128, provNum), #7 caratteri cinesi, #8 caratteri (7 + 1 speciale) per le tunisine
            )
            self.classifier2 = nn.Sequential(
                # nn.Dropout(),
                nn.Linear(53248, 128),
                # nn.ReLU(inplace=True),
                # nn.Dropout(),
                nn.Linear(128, alphaNum),
            )
            self.classifier3 = nn.Sequential(
                # nn.Dropout(),
                nn.Linear(53248, 128),
                # nn.ReLU(inplace=True),
                # nn.Dropout(),
                nn.Linear(128, adNum),
            )
            self.classifier4 = nn.Sequential(
                # nn.Dropout(),
                nn.Linear(53248, 128),
                # nn.ReLU(inplace=True),
                # nn.Dropout(),
                nn.Linear(128, adNum),
            )
            self.classifier5 = nn.Sequential(
                # nn.Dropout(),
                nn.Linear(53248, 128),
                # nn.ReLU(inplace=True),
                # nn.Dropout(),
                nn.Linear(128, adNum),
            )
            self.classifier6 = nn.Sequential(
                # nn.Dropout(),
                nn.Linear(53248, 128),
                # nn.ReLU(inplace=True),
                # nn.Dropout(),
                nn.Linear(128, adNum),
            )
            self.classifier7 = nn.Sequential(
                # nn.Dropout(),
                nn.Linear(53248, 128),
                # nn.ReLU(inplace=True),
                # nn.Dropout(),
                nn.Linear(128, adNum),
            )

        def load_wR2(self, path):
            self.wR2 = wR2_ada(numPoints)
            self.wR2 = torch.nn.DataParallel(self.wR2, device_ids=range(torch.cuda.device_count()))
            if not path is None:
                #with open(path, 'r') as f:
                self.wR2.load_state_dict(torch.load(path))
                # self.wR2 = self.wR2.cuda()
            # for param in self.wR2.parameters():
            #     param.requires_grad = False

        def forward(self, x):
            x0 = self.wR2.module.features[0](x)
            _x1 = self.wR2.module.features[1](x0)
            x2 = self.wR2.module.features[2](_x1)
            _x3 = self.wR2.module.features[3](x2)
            x4 = self.wR2.module.features[4](_x3)
            _x5 = self.wR2.module.features[5](x4)

            x6 = self.wR2.module.features[6](_x5)
            x7 = self.wR2.module.features[7](x6)
            x8 = self.wR2.module.features[8](x7)
            x9 = self.wR2.module.features[9](x8)
            x9 = x9.view(x9.size(0), -1)
            boxLoc = self.wR2.module.classifier(x9)

            h1, w1 = _x1.data.size()[2], _x1.data.size()[3]
            p1 = Variable(torch.FloatTensor([[w1,0,0,0],[0,h1,0,0],[0,0,w1,0],[0,0,0,h1]]).cuda(), requires_grad=False)
            h2, w2 = _x3.data.size()[2], _x3.data.size()[3]
            p2 = Variable(torch.FloatTensor([[w2,0,0,0],[0,h2,0,0],[0,0,w2,0],[0,0,0,h2]]).cuda(), requires_grad=False)
            h3, w3 = _x5.data.size()[2], _x5.data.size()[3]
            p3 = Variable(torch.FloatTensor([[w3,0,0,0],[0,h3,0,0],[0,0,w3,0],[0,0,0,h3]]).cuda(), requires_grad=False)

            # x, y, w, h --> x1, y1, x2, y2
            assert boxLoc.data.size()[1] == 4
            postfix = Variable(torch.FloatTensor([[1,0,1,0],[0,1,0,1],[-0.5,0,0.5,0],[0,-0.5,0,0.5]]).cuda(), requires_grad=False)
            boxNew = boxLoc.mm(postfix).clamp(min=0, max=1)

            # input = Variable(torch.rand(2, 1, 10, 10), requires_grad=True)
            # rois = Variable(torch.LongTensor([[0, 1, 2, 7, 8], [0, 3, 3, 8, 8], [1, 3, 3, 8, 8]]), requires_grad=False)
            roi1 = roi_pooling_ims(_x1, boxNew.mm(p1), size=(16, 8))
            roi2 = roi_pooling_ims(_x3, boxNew.mm(p2), size=(16, 8))
            roi3 = roi_pooling_ims(_x5, boxNew.mm(p3), size=(16, 8))
            rois = torch.cat((roi1, roi2, roi3), 1)

            _rois = rois.view(rois.size(0), -1)

            y0 = self.classifier1(_rois)
            y1 = self.classifier2(_rois)
            y2 = self.classifier3(_rois)
            y3 = self.classifier4(_rois)
            y4 = self.classifier5(_rois)
            y5 = self.classifier6(_rois)
            y6 = self.classifier7(_rois)
            return boxLoc, [y0, y1, y2, y3, y4, y5, y6]


In [4]:
model = wR2(4)
model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
model.load_state_dict(torch.load(r'C:\Users\marco\wR2_MishAdaBN\wR2_MishAdaBN.pth8'))
#model_conv = model_conv.cuda()

<All keys matched successfully>

In [33]:
print('Training these layers')
for name, param in model.named_parameters():
    #if param.requires_grad is True:
    if 'a' != name.split('.')[-1] and 'b' != name.split('.')[-1] and 'bn' != name.split('.')[-2]:
        param.requires_grad = False
    print(name, param.requires_grad)

Training these layers
module.features.0.0.weight False
module.features.0.0.bias False
module.features.0.1.a True
module.features.0.1.b True
module.features.0.1.bn.weight True
module.features.0.1.bn.bias True
module.features.1.0.weight False
module.features.1.0.bias False
module.features.1.1.a True
module.features.1.1.b True
module.features.1.1.bn.weight True
module.features.1.1.bn.bias True
module.features.2.0.weight False
module.features.2.0.bias False
module.features.2.1.a True
module.features.2.1.b True
module.features.2.1.bn.weight True
module.features.2.1.bn.bias True
module.features.3.0.weight False
module.features.3.0.bias False
module.features.3.1.a True
module.features.3.1.b True
module.features.3.1.bn.weight True
module.features.3.1.bn.bias True
module.features.4.0.weight False
module.features.4.0.bias False
module.features.4.1.a True
module.features.4.1.b True
module.features.4.1.bn.weight True
module.features.4.1.bn.bias True
module.features.5.0.weight False
module.features

In [6]:
child_counter = 0
for child in model.children():
    print("child", child_counter, "is:")
    print(child)
    child_counter += 1

child 0 is:
wR2(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 48, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
      (1): AdaptiveBatchNorm2d(
        (bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (2): Mish()
      (3): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Dropout(p=0.2, inplace=False)
    )
    (1): Sequential(
      (0): Conv2d(48, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): AdaptiveBatchNorm2d(
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (2): Mish()
      (3): MaxPool2d(kernel_size=2, stride=1, padding=1, dilation=1, ceil_mode=False)
      (4): Dropout(p=0.2, inplace=False)
    )
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): AdaptiveBatchNorm2d(
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affin

In [17]:
for child in model.children():
    for c in child.children():
        for d in c.children():
            for e in d.children():
                for param in e.parameters():
                    print(e, param.size())
        #break
    #break

Conv2d(3, 48, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)) torch.Size([48, 3, 5, 5])
Conv2d(3, 48, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)) torch.Size([48])
AdaptiveBatchNorm2d(
  (bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
) torch.Size([1, 1, 1, 1])
AdaptiveBatchNorm2d(
  (bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
) torch.Size([1, 1, 1, 1])
AdaptiveBatchNorm2d(
  (bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
) torch.Size([48])
AdaptiveBatchNorm2d(
  (bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
) torch.Size([48])
Conv2d(48, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2)) torch.Size([64, 48, 5, 5])
Conv2d(48, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2)) torch.Size([64])
AdaptiveBatchNorm2d(
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
) torch.Size([

In [3]:
    class wR2(nn.Module):
        def __init__(self, num_classes=1000):
            super(wR2, self).__init__()
            hidden1 = nn.Sequential(
                nn.Conv2d(in_channels=3, out_channels=48, kernel_size=5, padding=2, stride=2),
                nn.BatchNorm2d(num_features=48),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
                nn.Dropout(0.2)
            )
            hidden2 = nn.Sequential(
                nn.Conv2d(in_channels=48, out_channels=64, kernel_size=5, padding=2),
                nn.BatchNorm2d(num_features=64),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
                nn.Dropout(0.2)
            )
            hidden3 = nn.Sequential(
                nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, padding=2),
                nn.BatchNorm2d(num_features=128),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
                nn.Dropout(0.2)
            )
            hidden4 = nn.Sequential(
                nn.Conv2d(in_channels=128, out_channels=160, kernel_size=5, padding=2),
                nn.BatchNorm2d(num_features=160),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
                nn.Dropout(0.2)
            )
            hidden5 = nn.Sequential(
                nn.Conv2d(in_channels=160, out_channels=192, kernel_size=5, padding=2),
                nn.BatchNorm2d(num_features=192),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
                nn.Dropout(0.2)
            )
            hidden6 = nn.Sequential(
                nn.Conv2d(in_channels=192, out_channels=192, kernel_size=5, padding=2),
                nn.BatchNorm2d(num_features=192),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
                nn.Dropout(0.2)
            )
            hidden7 = nn.Sequential(
                nn.Conv2d(in_channels=192, out_channels=192, kernel_size=5, padding=2),
                nn.BatchNorm2d(num_features=192),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
                nn.Dropout(0.2)
            )
            hidden8 = nn.Sequential(
                nn.Conv2d(in_channels=192, out_channels=192, kernel_size=5, padding=2),
                nn.BatchNorm2d(num_features=192),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
                nn.Dropout(0.2)
            )
            hidden9 = nn.Sequential(
                nn.Conv2d(in_channels=192, out_channels=192, kernel_size=3, padding=1),
                nn.BatchNorm2d(num_features=192),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
                nn.Dropout(0.2)
            )
            hidden10 = nn.Sequential(
                nn.Conv2d(in_channels=192, out_channels=192, kernel_size=3, padding=1),
                nn.BatchNorm2d(num_features=192),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
                nn.Dropout(0.2)
            )
            self.features = nn.Sequential(
                hidden1,
                hidden2,
                hidden3,
                hidden4,
                hidden5,
                hidden6,
                hidden7,
                hidden8,
                hidden9,
                hidden10
            )
            self.classifier = nn.Sequential(
                nn.Linear(23232, 100),
                # nn.ReLU(inplace=True),
                nn.Linear(100, 100),
                # nn.ReLU(inplace=True),
                nn.Linear(100, num_classes),
            )

        def forward(self, x):
            x1 = self.features(x)
            x11 = x1.view(x1.size(0), -1)
            x = self.classifier(x11)
            return x


    class fh02(nn.Module):
        def __init__(self, num_points, num_classes, wrPath=None):
            super(fh02, self).__init__()
            self.load_wR2(wrPath)
            self.classifier1 = nn.Sequential(
                # nn.Dropout(),
                nn.Linear(53248, 128),
                # nn.ReLU(inplace=True),
                # nn.Dropout(),
                nn.Linear(128, provNum), #7 caratteri cinesi, #8 caratteri (7 + 1 speciale) per le tunisine
            )
            self.classifier2 = nn.Sequential(
                # nn.Dropout(),
                nn.Linear(53248, 128),
                # nn.ReLU(inplace=True),
                # nn.Dropout(),
                nn.Linear(128, alphaNum),
            )
            self.classifier3 = nn.Sequential(
                # nn.Dropout(),
                nn.Linear(53248, 128),
                # nn.ReLU(inplace=True),
                # nn.Dropout(),
                nn.Linear(128, adNum),
            )
            self.classifier4 = nn.Sequential(
                # nn.Dropout(),
                nn.Linear(53248, 128),
                # nn.ReLU(inplace=True),
                # nn.Dropout(),
                nn.Linear(128, adNum),
            )
            self.classifier5 = nn.Sequential(
                # nn.Dropout(),
                nn.Linear(53248, 128),
                # nn.ReLU(inplace=True),
                # nn.Dropout(),
                nn.Linear(128, adNum),
            )
            self.classifier6 = nn.Sequential(
                # nn.Dropout(),
                nn.Linear(53248, 128),
                # nn.ReLU(inplace=True),
                # nn.Dropout(),
                nn.Linear(128, adNum),
            )
            self.classifier7 = nn.Sequential(
                # nn.Dropout(),
                nn.Linear(53248, 128),
                # nn.ReLU(inplace=True),
                # nn.Dropout(),
                nn.Linear(128, adNum),
            )

        def load_wR2(self, path):
            self.wR2 = wR2(numPoints)
            self.wR2 = torch.nn.DataParallel(self.wR2, device_ids=range(torch.cuda.device_count()))
            if not path is None:
                #with open(path, 'r') as f:
                self.wR2.load_state_dict(torch.load(path))
                # self.wR2 = self.wR2.cuda()
            # for param in self.wR2.parameters():
            #     param.requires_grad = False

        def forward(self, x):
            x0 = self.wR2.module.features[0](x)
            _x1 = self.wR2.module.features[1](x0)
            x2 = self.wR2.module.features[2](_x1)
            _x3 = self.wR2.module.features[3](x2)
            x4 = self.wR2.module.features[4](_x3)
            _x5 = self.wR2.module.features[5](x4)

            x6 = self.wR2.module.features[6](_x5)
            x7 = self.wR2.module.features[7](x6)
            x8 = self.wR2.module.features[8](x7)
            x9 = self.wR2.module.features[9](x8)
            x9 = x9.view(x9.size(0), -1)
            boxLoc = self.wR2.module.classifier(x9)

            h1, w1 = _x1.data.size()[2], _x1.data.size()[3]
            p1 = Variable(torch.FloatTensor([[w1,0,0,0],[0,h1,0,0],[0,0,w1,0],[0,0,0,h1]]).cuda(), requires_grad=False)
            h2, w2 = _x3.data.size()[2], _x3.data.size()[3]
            p2 = Variable(torch.FloatTensor([[w2,0,0,0],[0,h2,0,0],[0,0,w2,0],[0,0,0,h2]]).cuda(), requires_grad=False)
            h3, w3 = _x5.data.size()[2], _x5.data.size()[3]
            p3 = Variable(torch.FloatTensor([[w3,0,0,0],[0,h3,0,0],[0,0,w3,0],[0,0,0,h3]]).cuda(), requires_grad=False)

            # x, y, w, h --> x1, y1, x2, y2
            assert boxLoc.data.size()[1] == 4
            postfix = Variable(torch.FloatTensor([[1,0,1,0],[0,1,0,1],[-0.5,0,0.5,0],[0,-0.5,0,0.5]]).cuda(), requires_grad=False)
            boxNew = boxLoc.mm(postfix).clamp(min=0, max=1)

            # input = Variable(torch.rand(2, 1, 10, 10), requires_grad=True)
            # rois = Variable(torch.LongTensor([[0, 1, 2, 7, 8], [0, 3, 3, 8, 8], [1, 3, 3, 8, 8]]), requires_grad=False)
            roi1 = roi_pooling_ims(_x1, boxNew.mm(p1), size=(16, 8))
            roi2 = roi_pooling_ims(_x3, boxNew.mm(p2), size=(16, 8))
            roi3 = roi_pooling_ims(_x5, boxNew.mm(p3), size=(16, 8))
            rois = torch.cat((roi1, roi2, roi3), 1)

            _rois = rois.view(rois.size(0), -1)

            y0 = self.classifier1(_rois)
            y1 = self.classifier2(_rois)
            y2 = self.classifier3(_rois)
            y3 = self.classifier4(_rois)
            y4 = self.classifier5(_rois)
            y5 = self.classifier6(_rois)
            y6 = self.classifier7(_rois)
            return boxLoc, [y0, y1, y2, y3, y4, y5, y6]

In [91]:
numPoints = 4
provNum, alphaNum, adNum = 38, 25, 35
model = fh02(4,4)
model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
model.load_state_dict(torch.load(r'C:\\Users\\marco\\rpnet\\fh02.pth'))

<All keys matched successfully>

In [120]:
for i, (name, param) in enumerate(model.named_parameters()):
    if i == 37:
        #break
        print(name, param)

module.wR2.module.features.9.0.bias Parameter containing:
tensor([ 7.7335e-03, -2.0678e-02,  2.0979e-02,  2.1724e-02,  1.4904e-02,
        -1.8673e-02, -7.0962e-03, -3.7591e-03,  1.7761e-02, -7.7613e-03,
         5.7872e-03, -1.0664e-02, -8.3345e-03, -1.1733e-02, -2.3360e-02,
        -1.9262e-02,  2.3450e-02, -4.2720e-03, -9.2829e-03,  8.7097e-03,
        -6.8697e-03,  4.2871e-03, -7.1529e-03, -9.4270e-03,  4.6189e-03,
        -1.7857e-02, -2.3621e-02, -9.0335e-03,  1.0020e-02,  4.3424e-03,
         4.1119e-03, -9.0928e-03, -1.8652e-02,  4.4880e-03,  1.6102e-03,
        -2.1328e-02, -8.1799e-03, -1.4946e-02, -6.5579e-03,  1.3557e-03,
        -1.8570e-03,  1.1557e-02, -4.2984e-03, -1.6316e-02, -9.8140e-03,
         5.1771e-03,  1.7007e-02,  1.3575e-02, -1.7333e-02,  2.0420e-02,
        -4.8040e-03, -2.3160e-02, -2.1747e-02, -2.2099e-02,  1.4349e-02,
        -1.1755e-02,  1.0399e-02,  4.4294e-03,  2.9721e-03,  1.5837e-02,
        -1.9071e-02,  2.2370e-02, -1.3439e-02, -2.0167e-02,  1.283

In [46]:
model_new = fh02_ada(7, 4)
model_new = torch.nn.DataParallel(model_new, device_ids=range(torch.cuda.device_count()))

In [58]:
for (name_new, param_new) in model_new.named_parameters():
    for (name, param) in model.named_parameters():
        string_new_param = ''.join(name_new.split('.')[:6]) + str(name_new.split('.')[-1])
        string_param = ''.join(name.split('.')[:6])+str(name.split('.')[-1])
        print(string_new_param, string_param)
        if  string_new_param == string_param:
            param_new.data.copy_(param.data)

modulewR2modulefeatures00weight modulewR2modulefeatures00weight
modulewR2modulefeatures00weight modulewR2modulefeatures00bias
modulewR2modulefeatures00weight modulewR2modulefeatures01weight
modulewR2modulefeatures00weight modulewR2modulefeatures01bias
modulewR2modulefeatures00weight modulewR2modulefeatures10weight
modulewR2modulefeatures00weight modulewR2modulefeatures10bias
modulewR2modulefeatures00weight modulewR2modulefeatures11weight
modulewR2modulefeatures00weight modulewR2modulefeatures11bias
modulewR2modulefeatures00weight modulewR2modulefeatures20weight
modulewR2modulefeatures00weight modulewR2modulefeatures20bias
modulewR2modulefeatures00weight modulewR2modulefeatures21weight
modulewR2modulefeatures00weight modulewR2modulefeatures21bias
modulewR2modulefeatures00weight modulewR2modulefeatures30weight
modulewR2modulefeatures00weight modulewR2modulefeatures30bias
modulewR2modulefeatures00weight modulewR2modulefeatures31weight
modulewR2modulefeatures00weight modulewR2modulefeature

modulewR2modulefeatures31a modulewR2modulefeatures21bias
modulewR2modulefeatures31a modulewR2modulefeatures30weight
modulewR2modulefeatures31a modulewR2modulefeatures30bias
modulewR2modulefeatures31a modulewR2modulefeatures31weight
modulewR2modulefeatures31a modulewR2modulefeatures31bias
modulewR2modulefeatures31a modulewR2modulefeatures40weight
modulewR2modulefeatures31a modulewR2modulefeatures40bias
modulewR2modulefeatures31a modulewR2modulefeatures41weight
modulewR2modulefeatures31a modulewR2modulefeatures41bias
modulewR2modulefeatures31a modulewR2modulefeatures50weight
modulewR2modulefeatures31a modulewR2modulefeatures50bias
modulewR2modulefeatures31a modulewR2modulefeatures51weight
modulewR2modulefeatures31a modulewR2modulefeatures51bias
modulewR2modulefeatures31a modulewR2modulefeatures60weight
modulewR2modulefeatures31a modulewR2modulefeatures60bias
modulewR2modulefeatures31a modulewR2modulefeatures61weight
modulewR2modulefeatures31a modulewR2modulefeatures61bias
modulewR2module

modulewR2modulefeatures61weight modulewR2modulefeatures71bias
modulewR2modulefeatures61weight modulewR2modulefeatures80weight
modulewR2modulefeatures61weight modulewR2modulefeatures80bias
modulewR2modulefeatures61weight modulewR2modulefeatures81weight
modulewR2modulefeatures61weight modulewR2modulefeatures81bias
modulewR2modulefeatures61weight modulewR2modulefeatures90weight
modulewR2modulefeatures61weight modulewR2modulefeatures90bias
modulewR2modulefeatures61weight modulewR2modulefeatures91weight
modulewR2modulefeatures61weight modulewR2modulefeatures91bias
modulewR2modulefeatures61weight modulewR2moduleclassifier0weightweight
modulewR2modulefeatures61weight modulewR2moduleclassifier0biasbias
modulewR2modulefeatures61weight modulewR2moduleclassifier1weightweight
modulewR2modulefeatures61weight modulewR2moduleclassifier1biasbias
modulewR2modulefeatures61weight modulewR2moduleclassifier2weightweight
modulewR2modulefeatures61weight modulewR2moduleclassifier2biasbias
modulewR2modulefeatu

modulewR2moduleclassifier0biasbias modulewR2modulefeatures31bias
modulewR2moduleclassifier0biasbias modulewR2modulefeatures40weight
modulewR2moduleclassifier0biasbias modulewR2modulefeatures40bias
modulewR2moduleclassifier0biasbias modulewR2modulefeatures41weight
modulewR2moduleclassifier0biasbias modulewR2modulefeatures41bias
modulewR2moduleclassifier0biasbias modulewR2modulefeatures50weight
modulewR2moduleclassifier0biasbias modulewR2modulefeatures50bias
modulewR2moduleclassifier0biasbias modulewR2modulefeatures51weight
modulewR2moduleclassifier0biasbias modulewR2modulefeatures51bias
modulewR2moduleclassifier0biasbias modulewR2modulefeatures60weight
modulewR2moduleclassifier0biasbias modulewR2modulefeatures60bias
modulewR2moduleclassifier0biasbias modulewR2modulefeatures61weight
modulewR2moduleclassifier0biasbias modulewR2modulefeatures61bias
modulewR2moduleclassifier0biasbias modulewR2modulefeatures70weight
modulewR2moduleclassifier0biasbias modulewR2modulefeatures70bias
modulewR2mo

moduleclassifier50biasbias modulewR2modulefeatures50weight
moduleclassifier50biasbias modulewR2modulefeatures50bias
moduleclassifier50biasbias modulewR2modulefeatures51weight
moduleclassifier50biasbias modulewR2modulefeatures51bias
moduleclassifier50biasbias modulewR2modulefeatures60weight
moduleclassifier50biasbias modulewR2modulefeatures60bias
moduleclassifier50biasbias modulewR2modulefeatures61weight
moduleclassifier50biasbias modulewR2modulefeatures61bias
moduleclassifier50biasbias modulewR2modulefeatures70weight
moduleclassifier50biasbias modulewR2modulefeatures70bias
moduleclassifier50biasbias modulewR2modulefeatures71weight
moduleclassifier50biasbias modulewR2modulefeatures71bias
moduleclassifier50biasbias modulewR2modulefeatures80weight
moduleclassifier50biasbias modulewR2modulefeatures80bias
moduleclassifier50biasbias modulewR2modulefeatures81weight
moduleclassifier50biasbias modulewR2modulefeatures81bias
moduleclassifier50biasbias modulewR2modulefeatures90weight
moduleclassif

In [59]:
for name_new, param_new in model_new.named_parameters():
    print(name_new, param_new)

module.wR2.module.features.0.0.weight Parameter containing:
tensor([[[[-7.3015e-02,  9.9613e-03, -7.9300e-02,  1.5014e-02, -4.8797e-02],
          [ 1.2601e-02,  7.8719e-02,  9.7223e-03,  1.3890e-01, -2.0339e-01],
          [-1.0421e-01,  5.3841e-02, -2.4982e-02,  1.9202e-01, -7.6844e-02],
          [-2.0843e-01, -6.3139e-02,  9.2212e-03, -7.7020e-03,  2.5191e-02],
          [-3.7392e-02, -5.6885e-02,  1.4358e-02,  1.9322e-01, -2.7947e-02]],

         [[-4.6864e-01, -3.1891e-02,  2.1757e-01,  3.4641e-01, -2.6679e-02],
          [-3.2070e-01,  1.7052e-02,  3.4284e-01,  2.3685e-01, -1.8310e-01],
          [-4.9052e-01,  1.5821e-01,  2.7820e-01,  1.9053e-01, -3.6257e-01],
          [-3.5351e-01,  5.3721e-02,  3.1068e-01,  2.6491e-01, -2.3272e-01],
          [-4.3857e-01,  2.5300e-01,  3.5088e-01,  1.6730e-01, -3.9931e-01]],

         [[-2.5741e-01, -7.3498e-02,  3.0470e-01,  3.2433e-01,  1.5448e-02],
          [-3.1683e-01,  1.4836e-01,  2.5910e-01,  1.3296e-01, -3.0274e-01],
          [-

module.wR2.module.features.2.0.weight Parameter containing:
tensor([[[[-4.1031e-02,  1.0505e-02,  9.1642e-02,  3.9884e-03, -6.1386e-02],
          [ 1.8762e-02,  1.1086e-01,  9.5687e-02, -7.8892e-03, -1.3168e-03],
          [-3.1453e-02,  7.9079e-02,  1.0347e-01, -4.7873e-02, -2.5763e-03],
          [-7.3909e-02, -5.0569e-02,  2.1305e-02,  3.6600e-02,  8.6335e-02],
          [-7.0370e-02, -4.7344e-02, -5.3262e-04,  1.1851e-01,  1.2740e-01]],

         [[-7.9226e-02, -3.3561e-02,  4.2956e-02,  1.5649e-01,  1.8518e-01],
          [-2.5796e-02, -2.0235e-02,  5.2346e-02,  1.7543e-01,  1.4585e-01],
          [-1.7170e-02, -1.9602e-02,  5.8148e-02,  4.4937e-02,  1.2779e-01],
          [ 1.3705e-02, -9.3586e-02, -3.2876e-02, -3.4549e-02,  1.0163e-02],
          [-2.6572e-02, -1.1622e-01, -1.1270e-01, -7.7962e-02, -2.9835e-02]],

         [[-5.1596e-02, -5.5094e-02, -5.8586e-02, -5.7772e-02, -5.0067e-02],
          [-1.2189e-02, -5.2433e-02, -3.6211e-02, -4.9943e-02, -3.9330e-02],
          [ 

module.wR2.module.features.4.0.weight Parameter containing:
tensor([[[[ 1.6506e-02,  2.2812e-02, -9.6774e-03, -1.8747e-02,  3.5191e-03],
          [ 5.3648e-03,  1.8271e-02, -1.8498e-02, -2.0202e-02, -1.9112e-02],
          [-8.6696e-04,  9.4014e-03, -1.2485e-02,  9.6651e-03,  8.5870e-03],
          [ 1.4992e-02,  6.2119e-03, -4.4243e-03,  4.0002e-03, -1.7696e-02],
          [ 1.0535e-02,  4.3832e-03, -1.2216e-02, -1.1015e-02, -1.3714e-02]],

         [[ 7.0344e-03, -1.3352e-02,  2.2254e-03,  4.7986e-03,  2.5341e-03],
          [-9.9042e-03,  2.4767e-03, -8.2293e-03,  1.1978e-02,  5.3906e-03],
          [ 8.4942e-03,  1.4306e-02,  1.3279e-02,  1.5699e-02, -3.3737e-03],
          [ 1.3754e-02, -1.1751e-02, -1.4415e-02,  1.0344e-02, -1.1152e-02],
          [-1.3435e-02,  6.8082e-03,  4.2776e-03,  4.1430e-03, -7.3563e-03]],

         [[-7.0009e-03,  3.2103e-04,  1.5062e-02, -4.6155e-03, -2.8847e-04],
          [-6.3022e-03,  9.2452e-03,  2.0387e-02,  1.0179e-02, -1.5787e-03],
          [-

module.wR2.module.features.6.0.weight Parameter containing:
tensor([[[[-1.1387e-02,  4.4486e-03, -5.5507e-03, -4.4268e-03, -5.1004e-04],
          [ 7.6133e-03, -6.2723e-03,  1.0699e-02, -7.9119e-05, -3.8384e-03],
          [-1.1605e-02, -1.5677e-03,  7.4533e-03, -9.8584e-04, -7.3047e-03],
          [-9.8364e-03,  1.1910e-02,  7.1417e-03, -1.6073e-03, -5.1724e-03],
          [-2.0204e-03, -4.8153e-04, -9.2787e-03,  2.6231e-03, -1.1185e-02]],

         [[ 1.3215e-02, -2.1013e-03, -1.4439e-02, -1.9353e-03, -3.1428e-03],
          [-2.3022e-03,  2.5424e-03,  5.0489e-03, -5.3357e-03, -8.4383e-03],
          [-1.4386e-02,  5.5245e-03, -3.5726e-03,  4.4450e-03,  5.4666e-03],
          [-6.8262e-03,  5.6887e-03,  5.7236e-03,  1.4737e-03,  1.0578e-02],
          [-3.7084e-03, -1.0828e-02, -2.3538e-03,  1.4338e-03, -1.1638e-02]],

         [[-1.3762e-02, -3.3898e-03,  8.7112e-03, -1.9298e-03, -6.9402e-03],
          [-1.3049e-02, -1.2592e-02,  3.5606e-03,  2.7428e-03, -1.4014e-02],
          [-

module.wR2.module.features.8.0.bias Parameter containing:
tensor([ 0.0044, -0.0106,  0.0038, -0.0156,  0.0189, -0.0238,  0.0116,  0.0185,
        -0.0141,  0.0216,  0.0111,  0.0143, -0.0095,  0.0204,  0.0183,  0.0113,
         0.0221, -0.0035, -0.0184, -0.0187,  0.0235,  0.0123, -0.0120,  0.0224,
         0.0024, -0.0232, -0.0028,  0.0039,  0.0133,  0.0022, -0.0234, -0.0204,
        -0.0216,  0.0041,  0.0039,  0.0069,  0.0084, -0.0041,  0.0032, -0.0169,
        -0.0060,  0.0060, -0.0161, -0.0078, -0.0154, -0.0176,  0.0003, -0.0136,
        -0.0135,  0.0115,  0.0053, -0.0140,  0.0239, -0.0200, -0.0010, -0.0076,
         0.0185, -0.0073, -0.0138,  0.0224,  0.0100, -0.0036, -0.0218, -0.0197,
        -0.0004, -0.0080, -0.0024, -0.0172,  0.0101, -0.0225, -0.0186,  0.0002,
         0.0177, -0.0200,  0.0199, -0.0118, -0.0227,  0.0055,  0.0148,  0.0004,
        -0.0192, -0.0106,  0.0229,  0.0050,  0.0224,  0.0139, -0.0206, -0.0142,
        -0.0066,  0.0238,  0.0225, -0.0153, -0.0164,  0.0049, 

module.classifier2.0.bias Parameter containing:
tensor([ 0.0339, -0.0298, -0.0037,  0.0017, -0.0099, -0.0703,  0.0087, -0.0611,
        -0.0257,  0.0403,  0.0245,  0.0223,  0.0062,  0.0422, -0.0077,  0.0059,
        -0.0134, -0.0342, -0.0028,  0.0180, -0.0048, -0.0267,  0.0141,  0.0035,
         0.0419,  0.0456,  0.0075,  0.0217, -0.0008,  0.0078, -0.0079,  0.0370,
        -0.0160, -0.0265, -0.0209,  0.0156,  0.0215,  0.0373, -0.0219,  0.0172,
         0.0039, -0.0091,  0.0448, -0.0112, -0.0282,  0.0219,  0.0131,  0.0237,
         0.0154, -0.0163, -0.0457, -0.0074, -0.0104, -0.0172,  0.0357,  0.0379,
         0.0300,  0.0238, -0.0018, -0.0444,  0.0587, -0.0348,  0.0275,  0.0295,
        -0.0386,  0.0513,  0.0345,  0.0060,  0.0159,  0.0065, -0.0208,  0.0072,
        -0.0155, -0.0082,  0.0351, -0.0213,  0.0404,  0.0426, -0.0053, -0.0137,
         0.0257,  0.0378,  0.0290, -0.0047,  0.0040,  0.0042, -0.0280,  0.0010,
         0.0306,  0.0022, -0.0070, -0.0177,  0.0561, -0.0431,  0.0483,  

In [60]:
torch.save(model_new.state_dict(), 'fh02_new.pth')

In [132]:
model_wr2 = wR2(4)
model_wr2 = torch.nn.DataParallel(model_wr2, device_ids=range(torch.cuda.device_count()))
model_wr2.load_state_dict(torch.load(r'C:\\Users\\marco\\rpnet\\wR2_base.pth'))

<All keys matched successfully>

In [130]:
for name_wr2, param_wr2 in model_wr2.named_parameters():
    print(name_wr2)

module.features.0.0.weight
module.features.0.0.bias
module.features.0.1.weight
module.features.0.1.bias
module.features.1.0.weight
module.features.1.0.bias
module.features.1.1.weight
module.features.1.1.bias
module.features.2.0.weight
module.features.2.0.bias
module.features.2.1.weight
module.features.2.1.bias
module.features.3.0.weight
module.features.3.0.bias
module.features.3.1.weight
module.features.3.1.bias
module.features.4.0.weight
module.features.4.0.bias
module.features.4.1.weight
module.features.4.1.bias
module.features.5.0.weight
module.features.5.0.bias
module.features.5.1.weight
module.features.5.1.bias
module.features.6.0.weight
module.features.6.0.bias
module.features.6.1.weight
module.features.6.1.bias
module.features.7.0.weight
module.features.7.0.bias
module.features.7.1.weight
module.features.7.1.bias
module.features.8.0.weight
module.features.8.0.bias
module.features.8.1.weight
module.features.8.1.bias
module.features.9.0.weight
module.features.9.0.bias
module.featu

In [133]:
for j, (name_wr2, param_wr2) in enumerate(model_wr2.named_parameters()):    
    for i, (name, param) in enumerate(model.named_parameters()):
        if i < 46 and j < 40:
            #break
            string_wr2 = ''.join(name_wr2.split('.')[:4]) + str(name_wr2.split('.')[-1])
            string_param = ''.join(name.split('.')[2:7])
            if  string_wr2 == string_param:
                print(torch.all(torch.eq(param_wr2.data, param.data)))
            elif i == j:
                print('skip', name)
                #param_wr2.data.copy_(param.data)
        elif j >= 40 and i < 46:
            string_param = '.'.join(name.split('.')[2:])
            #print(string_param, name_wr2)
            if name_wr2 == string_param:
                print(torch.all(torch.eq(param_wr2.data, param.data)))
            elif i == j:
                print('skip', name)
                #param_wr2.data.copy_(param.data)

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(Tru

In [108]:
for name_wr2, param_wr2 in model_wr2.named_parameters():
    print(name_wr2, param_wr2)

module.features.0.0.weight Parameter containing:
tensor([[[[-7.3015e-02,  9.9613e-03, -7.9300e-02,  1.5014e-02, -4.8797e-02],
          [ 1.2601e-02,  7.8719e-02,  9.7223e-03,  1.3890e-01, -2.0339e-01],
          [-1.0421e-01,  5.3841e-02, -2.4982e-02,  1.9202e-01, -7.6844e-02],
          [-2.0843e-01, -6.3139e-02,  9.2212e-03, -7.7020e-03,  2.5191e-02],
          [-3.7392e-02, -5.6885e-02,  1.4358e-02,  1.9322e-01, -2.7947e-02]],

         [[-4.6864e-01, -3.1891e-02,  2.1757e-01,  3.4641e-01, -2.6679e-02],
          [-3.2070e-01,  1.7052e-02,  3.4284e-01,  2.3685e-01, -1.8310e-01],
          [-4.9052e-01,  1.5821e-01,  2.7820e-01,  1.9053e-01, -3.6257e-01],
          [-3.5351e-01,  5.3721e-02,  3.1068e-01,  2.6491e-01, -2.3272e-01],
          [-4.3857e-01,  2.5300e-01,  3.5088e-01,  1.6730e-01, -3.9931e-01]],

         [[-2.5741e-01, -7.3498e-02,  3.0470e-01,  3.2433e-01,  1.5448e-02],
          [-3.1683e-01,  1.4836e-01,  2.5910e-01,  1.3296e-01, -3.0274e-01],
          [-3.4970e-01,

module.features.1.1.bias Parameter containing:
tensor([-0.4651, -0.3022, -0.5785, -0.3735,  0.0172, -0.2406, -0.4147, -0.5621,
        -0.4940, -0.1784, -0.2076, -0.1262, -0.2960, -0.2860, -0.0112, -0.0020,
        -0.2376, -0.1490, -0.2306, -0.1226, -0.2051, -0.6057, -0.0364, -0.3370,
        -0.0099, -0.4446, -0.2968, -0.0446, -0.3125, -0.4853, -0.2505, -0.4502,
        -0.1904, -0.2315, -0.0747, -0.3372, -0.0232, -0.3963, -0.3172, -0.3131,
        -0.4702, -0.3521, -0.4766, -0.6956, -0.3314, -0.2985, -0.3093, -0.1585,
        -0.1885, -0.0130, -0.0341, -0.1800, -0.2559, -0.4479, -0.5371, -0.2656,
        -0.0889, -0.3559, -0.4539, -0.2978, -0.5624,  0.0636, -0.4497, -0.2335],
       device='cuda:0', requires_grad=True)
module.features.2.0.weight Parameter containing:
tensor([[[[-4.1031e-02,  1.0505e-02,  9.1642e-02,  3.9884e-03, -6.1386e-02],
          [ 1.8762e-02,  1.1086e-01,  9.5687e-02, -7.8892e-03, -1.3168e-03],
          [-3.1453e-02,  7.9079e-02,  1.0347e-01, -4.7873e-02, -2

module.features.3.1.weight Parameter containing:
tensor([ 5.2600e-01, -3.7423e-05,  6.1855e-01,  5.2985e-01,  5.4741e-01,
         5.5889e-01,  5.2539e-02,  4.8774e-01,  4.5445e-01,  6.2039e-01,
         6.9806e-01,  6.7160e-01,  9.5532e-02,  3.6695e-01,  5.2297e-01,
         7.0199e-01,  6.8473e-01,  3.7993e-01,  1.9367e-01,  4.1881e-01,
         3.2350e-01,  5.6011e-01,  5.8172e-01,  4.1690e-01,  3.6745e-01,
         6.7032e-01,  6.4922e-01,  4.3329e-01,  4.5750e-01,  3.3296e-01,
         9.9532e-02,  6.0183e-01,  4.1659e-01,  6.1055e-01,  5.2638e-01,
         4.3728e-04,  6.3930e-01,  5.9922e-01,  4.0722e-01,  2.0948e-01,
         5.9523e-01,  4.3434e-01,  2.0756e-01,  5.5892e-01,  5.8886e-01,
         5.6346e-01,  3.8594e-01,  6.6267e-01,  6.2126e-01,  6.0431e-01,
         5.1026e-01,  5.2944e-01,  3.2288e-01,  5.1773e-01,  3.8383e-01,
         4.0329e-01,  3.8605e-01,  5.4723e-01,  4.4876e-01,  5.7727e-01,
         5.3845e-01,  3.1056e-01,  5.2127e-01,  3.7838e-01,  5.4394e-01,
  

module.features.5.0.bias Parameter containing:
tensor([-3.7544e-03, -3.9295e-04,  5.3161e-03, -4.4452e-03, -8.2798e-03,
        -1.1057e-02,  1.0803e-02, -7.8584e-03,  8.9109e-03,  2.0734e-03,
         9.6991e-03,  5.8386e-03, -3.8971e-03, -9.4548e-03,  9.6172e-03,
        -1.2734e-02,  8.8124e-03, -9.7102e-04, -7.9057e-03,  1.2019e-02,
         1.2939e-02, -1.0292e-02,  1.3449e-02,  8.4607e-03,  1.2248e-02,
        -1.3332e-02, -4.3939e-03, -1.1202e-03, -2.5916e-03,  4.9170e-03,
         1.3555e-02, -1.2155e-02,  2.2969e-03,  6.4186e-03, -7.2172e-03,
         6.7687e-03,  3.0897e-03,  9.4680e-03, -2.3054e-03, -1.2329e-02,
         4.5242e-04, -1.8908e-03, -3.9874e-03,  1.2852e-02, -6.0936e-03,
        -4.1765e-03,  1.1617e-02, -3.7428e-03, -5.3245e-03, -4.1380e-03,
         1.1018e-02,  8.7081e-03,  6.4843e-03,  1.1966e-02,  1.1871e-03,
         1.0987e-02,  4.5177e-03,  4.9346e-03,  1.2711e-02, -1.3336e-02,
        -1.1667e-02,  1.2957e-02,  7.8220e-03,  3.7657e-03,  9.9084e-03,
    

module.features.7.0.bias Parameter containing:
tensor([ 1.3096e-02,  5.5999e-03, -9.4481e-03,  1.7239e-03,  1.1241e-02,
         9.9598e-03,  5.4995e-03,  1.1972e-02, -1.3203e-02, -1.0007e-02,
         1.1498e-02,  2.6919e-03, -4.6463e-03,  4.2866e-03, -3.3668e-03,
        -5.3056e-03, -3.4418e-03, -1.1836e-02, -8.0809e-03,  2.6174e-03,
        -6.9844e-03, -9.0483e-03,  1.0059e-02, -7.7939e-03, -8.4916e-03,
         3.3496e-03, -1.3111e-02,  1.4119e-03, -2.2710e-03,  4.9309e-03,
        -4.3470e-03, -2.9684e-03,  6.9508e-03,  7.5314e-03,  1.3177e-02,
         4.5969e-03, -1.1104e-02, -6.0909e-04,  1.1235e-03,  9.4427e-03,
        -1.2683e-02, -1.0361e-02,  3.5238e-04, -5.0276e-03,  7.3321e-03,
         5.5648e-03,  1.3894e-02,  1.5439e-03,  7.2733e-03, -8.7758e-03,
        -5.0538e-03, -3.4703e-03, -4.9025e-03, -1.0750e-02, -9.5804e-03,
        -7.5108e-04,  1.1448e-02, -1.7352e-03,  1.8769e-03,  1.3074e-02,
        -8.4529e-03, -9.7437e-03, -9.9366e-03,  9.9582e-03, -1.5212e-03,
    

module.classifier.2.weight Parameter containing:
tensor([[ 5.1209e-03,  1.9917e-03, -8.5366e-03,  2.1177e-02,  1.2604e-02,
         -1.6645e-02,  5.5805e-03, -2.0722e-02, -8.7965e-03,  1.4199e-02,
         -2.7027e-02,  2.2609e-02,  3.6204e-03, -6.7930e-03, -9.8697e-03,
          3.0124e-02, -2.9555e-02,  1.7798e-02, -1.2293e-02, -1.9586e-02,
         -1.4350e-02,  2.3976e-02,  2.2322e-02, -8.6278e-04,  4.8695e-03,
         -9.2172e-03,  6.7924e-03,  2.5588e-02,  2.4769e-02, -2.9478e-02,
          3.0683e-02,  2.2130e-02,  1.3680e-02, -1.6177e-02,  4.9886e-03,
         -1.9210e-02, -4.1728e-02,  1.9907e-02, -3.0667e-03,  7.3732e-03,
          8.0672e-03,  1.1228e-02, -3.7285e-03,  2.0405e-02, -6.0692e-03,
          2.2869e-02, -3.4644e-03,  3.4421e-03,  3.0653e-02,  1.0507e-02,
          7.0423e-03, -2.4011e-02, -1.0431e-02,  2.4351e-02, -2.2817e-02,
          1.9184e-02,  1.7117e-02,  2.4176e-02,  2.5977e-02,  8.9444e-03,
          1.9089e-02,  8.0998e-03,  1.8559e-02,  2.7019e-02, -1

In [113]:
torch.save(model_wr2.state_dict(), 'wR2_ada2.pth')

In [82]:
model_ = wR2(4)
model_saved = torch.nn.DataParallel(model_saved, device_ids=range(torch.cuda.device_count()))
model_saved.load_state_dict(torch.load(r'C:\\Users\\marco\\rpnet\\wR2_base.pth'))

<All keys matched successfully>

In [83]:
for name_wr2, param_wr2 in model_saved.named_parameters():
    print(name_wr2, param_wr2)

module.features.0.0.weight Parameter containing:
tensor([[[[-7.3015e-02,  9.9613e-03, -7.9300e-02,  1.5014e-02, -4.8797e-02],
          [ 1.2601e-02,  7.8719e-02,  9.7223e-03,  1.3890e-01, -2.0339e-01],
          [-1.0421e-01,  5.3841e-02, -2.4982e-02,  1.9202e-01, -7.6844e-02],
          [-2.0843e-01, -6.3139e-02,  9.2212e-03, -7.7020e-03,  2.5191e-02],
          [-3.7392e-02, -5.6885e-02,  1.4358e-02,  1.9322e-01, -2.7947e-02]],

         [[-4.6864e-01, -3.1891e-02,  2.1757e-01,  3.4641e-01, -2.6679e-02],
          [-3.2070e-01,  1.7052e-02,  3.4284e-01,  2.3685e-01, -1.8310e-01],
          [-4.9052e-01,  1.5821e-01,  2.7820e-01,  1.9053e-01, -3.6257e-01],
          [-3.5351e-01,  5.3721e-02,  3.1068e-01,  2.6491e-01, -2.3272e-01],
          [-4.3857e-01,  2.5300e-01,  3.5088e-01,  1.6730e-01, -3.9931e-01]],

         [[-2.5741e-01, -7.3498e-02,  3.0470e-01,  3.2433e-01,  1.5448e-02],
          [-3.1683e-01,  1.4836e-01,  2.5910e-01,  1.3296e-01, -3.0274e-01],
          [-3.4970e-01,

module.features.2.0.weight Parameter containing:
tensor([[[[-4.1031e-02,  1.0505e-02,  9.1642e-02,  3.9884e-03, -6.1386e-02],
          [ 1.8762e-02,  1.1086e-01,  9.5687e-02, -7.8892e-03, -1.3168e-03],
          [-3.1453e-02,  7.9079e-02,  1.0347e-01, -4.7873e-02, -2.5763e-03],
          [-7.3909e-02, -5.0569e-02,  2.1305e-02,  3.6600e-02,  8.6335e-02],
          [-7.0370e-02, -4.7344e-02, -5.3262e-04,  1.1851e-01,  1.2740e-01]],

         [[-7.9226e-02, -3.3561e-02,  4.2956e-02,  1.5649e-01,  1.8518e-01],
          [-2.5796e-02, -2.0235e-02,  5.2346e-02,  1.7543e-01,  1.4585e-01],
          [-1.7170e-02, -1.9602e-02,  5.8148e-02,  4.4937e-02,  1.2779e-01],
          [ 1.3705e-02, -9.3586e-02, -3.2876e-02, -3.4549e-02,  1.0163e-02],
          [-2.6572e-02, -1.1622e-01, -1.1270e-01, -7.7962e-02, -2.9835e-02]],

         [[-5.1596e-02, -5.5094e-02, -5.8586e-02, -5.7772e-02, -5.0067e-02],
          [-1.2189e-02, -5.2433e-02, -3.6211e-02, -4.9943e-02, -3.9330e-02],
          [ 7.1760e-03,

module.features.4.0.bias Parameter containing:
tensor([-0.0025,  0.0145,  0.0137,  0.0157,  0.0108, -0.0147,  0.0087, -0.0006,
        -0.0087, -0.0034,  0.0155, -0.0023,  0.0068, -0.0075,  0.0119,  0.0032,
        -0.0073,  0.0115, -0.0058,  0.0029,  0.0051,  0.0109,  0.0088,  0.0149,
         0.0140,  0.0118,  0.0075, -0.0091,  0.0048,  0.0143, -0.0064, -0.0068,
        -0.0090,  0.0127, -0.0044,  0.0039, -0.0001, -0.0131,  0.0153, -0.0140,
         0.0133,  0.0055, -0.0125, -0.0075, -0.0127,  0.0061,  0.0045,  0.0052,
        -0.0063, -0.0122,  0.0089, -0.0153,  0.0078, -0.0112,  0.0087,  0.0007,
         0.0019, -0.0083,  0.0087,  0.0081, -0.0097,  0.0057,  0.0152, -0.0036,
         0.0123, -0.0040, -0.0032, -0.0145,  0.0094, -0.0020, -0.0127,  0.0142,
         0.0079, -0.0060,  0.0056, -0.0116,  0.0098,  0.0043,  0.0144,  0.0068,
         0.0094, -0.0155,  0.0063, -0.0132,  0.0014,  0.0090, -0.0034,  0.0023,
        -0.0016,  0.0123, -0.0013, -0.0113,  0.0066, -0.0036, -0.0095,  0

module.features.6.0.weight Parameter containing:
tensor([[[[-1.1387e-02,  4.4486e-03, -5.5507e-03, -4.4268e-03, -5.1004e-04],
          [ 7.6133e-03, -6.2723e-03,  1.0699e-02, -7.9119e-05, -3.8384e-03],
          [-1.1605e-02, -1.5677e-03,  7.4533e-03, -9.8584e-04, -7.3047e-03],
          [-9.8364e-03,  1.1910e-02,  7.1417e-03, -1.6073e-03, -5.1724e-03],
          [-2.0204e-03, -4.8153e-04, -9.2787e-03,  2.6231e-03, -1.1185e-02]],

         [[ 1.3215e-02, -2.1013e-03, -1.4439e-02, -1.9353e-03, -3.1428e-03],
          [-2.3022e-03,  2.5424e-03,  5.0489e-03, -5.3357e-03, -8.4383e-03],
          [-1.4386e-02,  5.5245e-03, -3.5726e-03,  4.4450e-03,  5.4666e-03],
          [-6.8262e-03,  5.6887e-03,  5.7236e-03,  1.4737e-03,  1.0578e-02],
          [-3.7084e-03, -1.0828e-02, -2.3538e-03,  1.4338e-03, -1.1638e-02]],

         [[-1.3762e-02, -3.3898e-03,  8.7112e-03, -1.9298e-03, -6.9402e-03],
          [-1.3049e-02, -1.2592e-02,  3.5606e-03,  2.7428e-03, -1.4014e-02],
          [-2.0635e-03,

module.features.8.0.weight Parameter containing:
tensor([[[[ 0.0113,  0.0042, -0.0070],
          [ 0.0168,  0.0088, -0.0048],
          [-0.0150,  0.0109,  0.0010]],

         [[ 0.0213,  0.0104, -0.0052],
          [-0.0123,  0.0025, -0.0183],
          [-0.0004, -0.0058, -0.0017]],

         [[ 0.0126, -0.0008, -0.0183],
          [-0.0236, -0.0012, -0.0002],
          [-0.0174, -0.0249,  0.0184]],

         ...,

         [[ 0.0192, -0.0062,  0.0144],
          [-0.0005, -0.0204, -0.0032],
          [ 0.0104, -0.0218, -0.0106]],

         [[ 0.0078,  0.0099,  0.0087],
          [ 0.0145, -0.0016,  0.0074],
          [ 0.0114,  0.0101, -0.0097]],

         [[-0.0014,  0.0081,  0.0053],
          [-0.0223,  0.0012,  0.0168],
          [ 0.0188,  0.0095, -0.0178]]],


        [[[ 0.0020,  0.0215,  0.0113],
          [ 0.0105,  0.0231, -0.0158],
          [ 0.0186,  0.0159,  0.0149]],

         [[ 0.0098, -0.0184,  0.0025],
          [-0.0144,  0.0216,  0.0237],
          [ 0.0191,  0.

module.classifier.2.bias Parameter containing:
tensor([ 0.0262,  0.0169,  0.0804, -0.0353], device='cuda:0',
       requires_grad=True)
